In [1]:
import os

def get_file_list(folder_path):
    """
    지정된 폴더 내 모든 파일 이름을 리스트로 반환합니다.
    폴더가 존재하지 않거나 파일이 없을 경우 빈 리스트를 반환합니다.
    """
    if not os.path.isdir(folder_path):
        print("유효한 폴더 경로가 아닙니다.")
        return []

    return [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

In [10]:
folder_path = "../00_data/안전_가이드"
files = get_file_list(folder_path=folder_path)
files

['안전보건 실무길잡이_선박건조 및 수리업.pkl']

In [ ]:
import pickle

documents = []
for file in files:
    file_path = f"{folder_path}/{file}"
    # 파일 열기 및 데이터 로드
    with open(file_path, 'rb') as f:
        document = pickle.load(f)
        # document[0].metadata["lv1_cat"] = "안전"
        # document[0].metadata["lv2_cat"] = "가이드"
        documents.append(document)

# 데이터 확인
print(len(documents))
print(documents)

1
[[Document(metadata={'filename': '안전보건 실무길잡이_선박건조 및 수리업.pdf', 'page': '0', 'lv1_cat': '안전', 'lv2_cat': '가이드'}, page_content='This page explains 안전보건 실무길잡이_선박건조 및 수리업 that belongs to 안전 and  가이드 categories.\n안전보건 실무길잡이\n\n선박건조 및 수리업\n\n<!-- image -->\n\n산업재해예방\n\n안전보건공단\n\n<!-- image -->'), Document(metadata={'filename': '안전보건 실무길잡이_선박건조 및 수리업.pdf', 'page': '1'}, page_content='This page explains 안전보건 실무길잡이_선박건조 및 수리업 that belongs to 안전 and  가이드 categories.\n안전보건 실무길잡이\n\n## 선박건조 및 수리업'), Document(metadata={'filename': '안전보건 실무길잡이_선박건조 및 수리업.pdf', 'page': '2'}, page_content='This page explains 안전보건 실무길잡이_선박건조 및 수리업 that belongs to 안전 and  가이드 categories.\n## 안전보건 실무길잡이 선박건조 및 수리업\n\n2020 교육홍보본부 716\n\n발행일\n\n2011년 12월 16일 초판\n\n2020년 11월 30일 2쇄 개정판\n\n발행인\n\n박두용\n\n발행처\n\n안전보건공단 울산광역시 중구 종가로 400 Tel 052.7030.500\n\n집 필\n\n안전보건공단 김종운\n\n개정검토\n\n안전보건공단 이용근\n\n교육홍보본부 고광재, 김종환, 박준형\n\n편집디자인\n\n필드가이드 Tel 02.6375.2665\n\n이교재는 안전보건공단의 동의 없이 무단으로 수정 편집하거나 이틀 활용하여 다른 저작물올 제작하는 것은 저작권번에 위반되는 것이므

In [12]:
from langchain_ollama import OllamaEmbeddings
from langchain_elasticsearch import ElasticsearchStore, DenseVectorStrategy

def init_elastic_vectorstore(index_name:str):
    vector_store = ElasticsearchStore(
        index_name=index_name, 
        embedding=OllamaEmbeddings(base_url="http://localhost:11434", model="bge-m3:latest"), 
        es_url="http://localhost:9200",
        es_user="Kstyle",
        es_password="12345",
        # strategy=DenseVectorStrategy(hybrid=True)
        )
    return vector_store

In [13]:
from elasticsearch import Elasticsearch

def get_elastic_index_list():
    es = Elasticsearch(["http://localhost:9200"],
                    basic_auth=('Kstyle', '12345'))
    # Get all index names
    response = es.cat.indices(index='*', format='json')
    return response

index_list = get_elastic_index_list()
index_list

ListApiResponse([{'health': 'yellow', 'status': 'open', 'index': 'ship_safety', 'uuid': 'E-OwakNWSj-oN8IjNZ6ksQ', 'pri': '1', 'rep': '1', 'docs.count': '457', 'docs.deleted': '0', 'store.size': '7.6mb', 'pri.store.size': '7.6mb', 'dataset.size': '7.6mb'}])

In [14]:
def delete_index(index_name:str):
    es = Elasticsearch(["http://localhost:9200"],
                   basic_auth=('Kstyle', '12345'))
    response = es.indices.delete(index=index_name)
    print(f"Index '{index_name}' deletion response:", response)

# delete_index(index_name="ship_safety")

In [15]:
index_name="ship_safety"
vector_store = init_elastic_vectorstore(index_name=index_name)
vector_store

In [16]:
from tqdm.auto import tqdm
from uuid import uuid4
from langchain_core.documents import Document

for doc in tqdm(documents):
    uuids = [str(uuid4()) for _ in range(len(doc))]
    vector_store.add_documents(documents=doc, ids=uuids)

100%|██████████████████████████████████████████████████████████████████████████████████████| 1/1 [17:41<00:00, 1061.10s/it]


In [17]:
index_list = get_elastic_index_list()
index_list

ListApiResponse([{'health': 'yellow', 'status': 'open', 'index': 'ship_safety', 'uuid': 'E-OwakNWSj-oN8IjNZ6ksQ', 'pri': '1', 'rep': '1', 'docs.count': '694', 'docs.deleted': '0', 'store.size': '10.9mb', 'pri.store.size': '10.9mb', 'dataset.size': '10.9mb'}])

In [18]:
import requests

url = 'http://127.0.0.1:9200/_cat/indices?v'
response = requests.get(url)

if response.status_code == 200:
    print(response.text)
else:
    print("Failed to retrieve indices. Status code: {response.status_code}")

health status index       uuid                   pri rep docs.count docs.deleted store.size pri.store.size dataset.size
yellow open   ship_safety E-OwakNWSj-oN8IjNZ6ksQ   1   1        694            0     10.9mb         10.9mb       10.9mb

